In [1]:
#!/usr/bin/env python
"""
Demo pipeline for processing voltage imaging data. The processing pipeline
includes motion correction, memory mapping, segmentation, denoising and source
extraction. The demo shows how to construct the params, MotionCorrect and VOLPY 
objects and call the relevant functions. See inside for detail.
Dataset courtesy of Karel Svoboda Lab (Janelia Research Campus).
author: @caichangjia
"""
import cv2
import glob
import h5py
import logging
import matplotlib.pyplot as plt
import numpy as np
import os



#os.environ["MESMERIZE_N_PROCESSES"] = '40'

try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.paths import caiman_datadir
from caiman.source_extraction.volpy import utils
from caiman.source_extraction.volpy.volparams import volparams
from caiman.source_extraction.volpy.volpy import VOLPY
from caiman.summary_images import local_correlations_movie_offline
from caiman.summary_images import mean_image
from caiman.utils.utils import download_demo, download_model

# %%
# Set up the logger (optional); change this if you like.
# You can log to a file using the filename parameter, or make the output more
# or less verbose by setting level to logging.DEBUG, logging.INFO,
# logging.WARNING, or logging.ERROR
logging.basicConfig(format=
                    "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s]" \
                    "[%(process)d] %(message)s",
                    level=logging.INFO)


In [2]:
def bulk_MC(filepath,save_name):
    print("bulk")
    
    fnames = os.fsdecode(filepath)
    file_dir = os.path.split(fnames)[0]
    
    # Setup some parameters for data and motion correction dataset parameters
    fr = 600                                     # sample rate of the movie
    ROIs = None                                     # Region of interests
    index = None                                    # index of neurons
    weights = None                                  # reuse spatial weights by 
                                                    # opts.change_params(params_dict={'weights':vpy.estimates['weights']})
    # Motion correction parameters
    pw_rigid = False                                # flag for pw-rigid motion correction
    gSig_filt = (6, 6)                              # size of filter, in general gSig (see below),
                                                    # change this one if algorithm does not work
    max_shifts = (15, 15)                             # maximum allowed rigid shift
    strides = (48, 48)                              # start a new patch for pw-rigid motion correction every x pixels
    overlaps = (24, 24)                             # overlap between pathes (size of patch strides+overlaps)
    max_deviation_rigid = 3                         # maximum deviation allowed for patch with respect to rigid shifts
    border_nan = 'copy'

    opts_dict = {
        'fnames': fnames,
        'fr': fr,
        'index': index,
        'ROIs': ROIs,
        'weights': weights,
        'pw_rigid': pw_rigid,
        'max_shifts': max_shifts,
        'gSig_filt': gSig_filt,
        'strides': strides,
        'overlaps': overlaps,
        'max_deviation_rigid': max_deviation_rigid,
        'border_nan': border_nan
    }

    opts = volparams(params_dict=opts_dict)

    if 'dview' in locals():
        cm.stop_server(dview=dview)

    c, dview, n_processes = cm.cluster.setup_cluster(
            backend='local', n_processes=50, single_thread=False)

    print("Motion COrrection")
    # %%% MOTION CORRECTION
    # first we create a motion correction object with the specified parameters
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    dview.terminate()

    # Save MC file
    m_rig = cm.load(mc.mmap_file)
    m_rig.save(save_name+'_mc_.tif')




In [5]:


filepath = 'SIM_GCAMP\\SIM_GC8_1000_6_p3.tif'
save_name = 'SIM_GCAMP\\SIM_GC8_1000_6_p3_mc'
bulk_MC(filepath,save_name)

      262613 [volparams.py:                 set():95][26072] Changing key fnames in group data from None to SIM_GCAMP\SIM_GC8_1000_6_p3.tif
      262613 [volparams.py:                 set():95][26072] Changing key fr in group data from None to 600
      262614 [volparams.py:                 set():95][26072] Changing key max_shifts in group motion from (6, 6) to (15, 15)
      262614 [volparams.py:                 set():95][26072] Changing key gSig_filt in group motion from None to (6, 6)
      262615 [volparams.py:                 set():95][26072] Changing key strides in group motion from (96, 96) to (48, 48)
      262616 [volparams.py:                 set():95][26072] Changing key overlaps in group motion from (32, 32) to (24, 24)


bulk
Motion COrrection


      263337 [movies.py:      extract_shifts():343][26072] Movie average is negative. Removing 1st percentile.
      263339 [movies.py:      extract_shifts():361][26072] Movie average is negative. Removing 1st percentile.
      263402 [movies.py:      extract_shifts():343][26072] Movie average is negative. Removing 1st percentile.
      263404 [movies.py:      extract_shifts():361][26072] Movie average is negative. Removing 1st percentile.
      263466 [movies.py:      extract_shifts():343][26072] Movie average is negative. Removing 1st percentile.
      263468 [movies.py:      extract_shifts():361][26072] Movie average is negative. Removing 1st percentile.
      263547 [motion_correction.py:motion_correction_piecewise():3197][26072] Saving file as SIM_GCAMP\SIM_GC8_1000_6_p3_rig__d1_126_d2_298_d3_1_order_F_frames_20000.mmap
      263549 [motion_correction.py:motion_correction_piecewise():3210][26072] ** Starting parallel motion correction **
      286272 [motion_correction.py:motion_c

Decode mmap filename SIM_GCAMP\SIM_GC8_1000_6_p3_rig__d1_126_d2_298_d3_1_order_F_frames_20000.mmap


100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


In [ ]:
def signal_filter(sg, freq, fr, order=3, mode='high'):
    from scipy import signal
    """
    Function for high/low passing the signal with butterworth filter
    
    Args:
        sg: 1-d array
            input signal
            
        freq: float
            cutoff frequency
        
        order: int
            order of the filter
        
        mode: str
            'high' for high-pass filtering, 'low' for low-pass filtering
            
    Returns:
        sg: 1-d array
            signal after filtering            
    """
    normFreq = freq / (fr / 2)
    b, a = signal.butter(order, normFreq, mode)
    sg = np.single(signal.filtfilt(b, a, sg, padtype='odd', padlen=3 * (max(len(b), len(a)) - 1)))

    return sg

In [ ]:
def bulk_Detrend(filepath,savename):
    print("detrend")
    
    

In [3]:
fnames = r"061523"



In [5]:
def process_Multiple_Files(folder_Name):
    directory = os.fsencode(folder_Name)
    for folder in os.listdir(directory):
        subdirectory = os.fsencode(folder)
        print(subdirectory)


        for subsub in os.listdir(directory+b'\\'+subdirectory):
            subsubdirectory = os.fsencode(subsub)
            for file in os.listdir(directory+b'\\'+subdirectory+b'\\'+subsubdirectory):


                filename = os.fsdecode(file)
                if filename.endswith(".tif"):
                    bulk_MC(directory+b'\\' + subdirectory + b'\\' + subsubdirectory + b'\\' + os.fsencode(filename),os.fsdecode(subdirectory))
                    #print(file)
                    continue
                else:
                    #print(file)
                    continue
        

In [22]:
def process_Multiple_Files_1level(folder_Name):
    directory = os.fsencode(folder_Name)

    for file in os.listdir(directory):


                filename = os.fsdecode(file)
                if filename.endswith(".tif"):
                    bulk_MC(directory+  b'\\' + os.fsencode(filename),os.fsdecode(filename)+'_MC')
                    #print(file)
                    continue
                else:
                    #print(file)
                    continue

In [6]:
fnames = r"073123\\RC1_SV"
process_Multiple_Files(fnames)

       45239 [volparams.py:                 set():95][27036] Changing key fnames in group data from None to 073123\\RC1_SV\1_415_b4.tif\_1\_1_MMStack_Pos0.ome.tif
       45240 [volparams.py:                 set():95][27036] Changing key fr in group data from None to 417
       45240 [volparams.py:                 set():95][27036] Changing key max_shifts in group motion from (6, 6) to (15, 15)
       45241 [volparams.py:                 set():95][27036] Changing key gSig_filt in group motion from None to (6, 6)
       45242 [volparams.py:                 set():95][27036] Changing key strides in group motion from (96, 96) to (48, 48)
       45242 [volparams.py:                 set():95][27036] Changing key overlaps in group motion from (32, 32) to (24, 24)


b'1_415_b4.tif'
bulk
Motion COrrection


       47739 [movies.py:      extract_shifts():343][27036] Movie average is negative. Removing 1st percentile.
       47741 [movies.py:      extract_shifts():361][27036] Movie average is negative. Removing 1st percentile.
       47817 [movies.py:      extract_shifts():343][27036] Movie average is negative. Removing 1st percentile.
       47819 [movies.py:      extract_shifts():361][27036] Movie average is negative. Removing 1st percentile.
       47893 [movies.py:      extract_shifts():343][27036] Movie average is negative. Removing 1st percentile.
       47895 [movies.py:      extract_shifts():361][27036] Movie average is negative. Removing 1st percentile.
       48671 [motion_correction.py:motion_correction_piecewise():3197][27036] Saving file as 073123\\RC1_SV\1_415_b4.tif\_1\_1_MMStack_Pos0.ome_rig__d1_136_d2_311_d3_1_order_F_frames_12000.mmap
       48688 [motion_correction.py:motion_correction_piecewise():3210][27036] ** Starting parallel motion correction **
       65653 [motion

Decode mmap filename 073123\\RC1_SV\1_415_b4.tif\_1\_1_MMStack_Pos0.ome_rig__d1_136_d2_311_d3_1_order_F_frames_12000.mmap


100%|██████████| 1/1 [00:01<00:00,  1.21s/it]
       71909 [volparams.py:                 set():95][27036] Changing key fnames in group data from None to 073123\\RC1_SV\3_417_b4\_1\_1_MMStack_Pos0.ome.tif
       71910 [volparams.py:                 set():95][27036] Changing key fr in group data from None to 417
       71910 [volparams.py:                 set():95][27036] Changing key max_shifts in group motion from (6, 6) to (15, 15)
       71911 [volparams.py:                 set():95][27036] Changing key gSig_filt in group motion from None to (6, 6)
       71912 [volparams.py:                 set():95][27036] Changing key strides in group motion from (96, 96) to (48, 48)
       71912 [volparams.py:                 set():95][27036] Changing key overlaps in group motion from (32, 32) to (24, 24)


...

Decode mmap filename 073123\\RC1_SV\8_400_b4\_1\_1_MMStack_Pos0.ome_rig__d1_119_d2_360_d3_1_order_F_frames_12000.mmap


100%|██████████| 1/1 [00:01<00:00,  1.50s/it]
      268280 [volparams.py:                 set():95][27036] Changing key fnames in group data from None to 073123\\RC1_SV\9_417_b2\_1\_1_MMStack_Pos0.ome.tif
      268281 [volparams.py:                 set():95][27036] Changing key fr in group data from None to 417
      268282 [volparams.py:                 set():95][27036] Changing key max_shifts in group motion from (6, 6) to (15, 15)
      268283 [volparams.py:                 set():95][27036] Changing key gSig_filt in group motion from None to (6, 6)
      268283 [volparams.py:                 set():95][27036] Changing key strides in group motion from (96, 96) to (48, 48)
      268284 [volparams.py:                 set():95][27036] Changing key overlaps in group motion from (32, 32) to (24, 24)


b'9_417_b2'
bulk
Motion COrrection


      268992 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read '_1_MMStack_Pos0_1.ome.tif'. Missing data are zeroed
      269367 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read 1078 files
      271468 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read '_1_MMStack_Pos0_1.ome.tif'. Missing data are zeroed
      271833 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read 1078 files
      272304 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read '_1_MMStack_Pos0_1.ome.tif'. Missing data are zeroed
      272669 [tifffile.py:         log_warning():22909][27036] <tifffile.TiffFile '_1_MMStack_Pos0.ome.tif'> OME series failed to read 1078 file

IndexError: index 9526 out of range(9526)